In [16]:
import math
from queue import PriorityQueue

def calculate_euclidean_distance(coord1, coord2):
    x1, y1, z1 = coord1
    x2, y2, z2 = coord2
    dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2 + (z2 - z1) ** 2)
    return int(dist)

def read_distance(file_path):
    graph = Graph()
    with open(file_path, "r") as f:
        lines = f.readlines()
        for line in lines:
            source, destination, distance = line.split(",")
            graph.addEdge(source, destination, int(distance))
    return graph


def read_coordinates(file_path, goal, a, b, c):
    with open(file_path, "r") as f:
        lines = f.readlines()
    for line in lines[1:]:
        node, x, y, z = line.strip().split(",")
        if node == goal:
            return float(x), float(y), float(z)


class Graph:
    def __init__(self):
        self.graph = {}

    def addEdge(self, source, destination, distance):
        if source not in self.graph:
            self.graph[source] = []
        if destination not in self.graph:
            self.graph[destination] = []

        self.graph[source].append([destination, distance])
        self.graph[destination].append([source, distance])

    def print_graph(self):
        for key in self.graph:
            print(f"{key}\t->\t{self.graph[key]}")

    def get_path(self, parent, node, g_scores):
        path = [node]
        total_cost = 0
        while parent[node] is not None:
            total_cost += g_scores[node]
            node = parent[node]
            path.append(node)
        return list(reversed(path)), total_cost

    def dijkstra(self, start, goal):
        pq = PriorityQueue()
        pq.put((0, start))

        parent = {start: None}
        visited = set()
        distances = {node: float('inf') for node in self.graph}
        distances[start] = 0

        while not pq.empty():
            current_node = pq.get()[1]

            if current_node == goal:
                return self.get_path(parent, current_node, distances)
            if current_node in visited:
                continue
            visited.add(current_node)
            for neighbor in self.graph[current_node]:
                alt_distance = distances[current_node] + neighbor[1]
                if alt_distance < distances[neighbor[0]]:
                    distances[neighbor[0]] = alt_distance
                    parent[neighbor[0]] = current_node
                    pq.put((distances[neighbor[0]], neighbor[0]))

        return None

    def goal_vertices(self, goal, a, b, c):
        with open("Coordinates.csv", "r") as f:
            lines = f.readlines()
        for line in lines[1:]:
            node, x, y, z = line.strip().split(",")
            if node == goal:
                return float(x), float(y), float(z)

    def a_star(self, start, goal, heuristic):
        pq = PriorityQueue()
        f_score = {start: 0 + heuristic[start]}
        pq.put((f_score[start], start))

        parent = {start: None}
        visited = set()
        g_score = {start: 0}

        while not pq.empty():
            current_node = pq.get()[1]

            if current_node == goal:
                return self.get_path(parent, current_node, g_score)
            if current_node in visited:
                continue
            visited.add(current_node)
            for neighbor in self.graph[current_node]:
                if neighbor[0] not in visited:
                    parent[neighbor[0]] = current_node
                    g_score[neighbor[0]] = g_score[parent[neighbor[0]]] + neighbor[1]
                    f_score[neighbor[0]] = g_score[neighbor[0]] + heuristic[neighbor[0]]
                    pq.put((f_score[neighbor[0]], neighbor[0]))

        return None, float('inf')


def main():
    g = read_distance("distances.csv")
    g.print_graph()

    start = input("Enter source name: ")
    goal = input("Enter Destination name: ")

    vala, valb, valc = read_coordinates("Coordinates.csv", goal, 0, 0, 0)
    hn = {}

    with open("Coordinates.csv", "r") as f:
        lines = f.readlines()
    for line in lines[1:]:
        node, x, y, z = line.strip().split(",")
        x, y, z = map(float, (x, y, z))
        coord_goal = (vala, valb, valc)
        coord_node = (x, y, z)
        dist = calculate_euclidean_distance(coord_goal, coord_node)
        hn[node] = int(dist)

    path_a_star, cost_a_star = g.a_star(start, goal, hn)
    print(f"A* Path: {path_a_star}, Total Cost of the Path: {cost_a_star}")

    path_dijkstra, cost_dijkstra = g.dijkstra(start, goal)
    print(f"Dijkstra Path: {path_dijkstra}, Total Cost of the Path: {cost_dijkstra}")

    if path_a_star == path_dijkstra and cost_a_star == cost_dijkstra:
        print("Same path and total cost.")
    else:
        print("Different paths and/or total costs.")


if __name__ == "__main__":
    main()

Sun	->	[['Proxima Centauri', 643], ['Lalande 21185', 1059], ['Lacaille 9352', 1194], ["Luyten's Star", 1251], ['YZ Ceti', 1766], ['Tau Ceti', 3042], ['Gliese 1061', 2773], ['Wolf 1061', 2501], ['Gliese 876', 2710], ['82 G. Eridani', 3247], ['Gliese 581', 2306], ['Proxima Centauri', 711], ['Lalande 21185', 988], ['Lacaille 9352', 1203], ["Luyten's Star", 1312], ['YZ Ceti', 1791], ['Tau Ceti', 3021], ['Gliese 1061', 2762], ['Wolf 1061', 2511], ['Gliese 876', 2722], ['82 G. Eridani', 3130], ['Gliese 581', 2344], ['TRAPPIST-1', 10480]]
Proxima Centauri	->	[['Sun', 643], ['Sun', 711], ['Lalande 21185', 558], ['Lacaille 9352', 1357], ["Luyten's Star", 1351], ['YZ Ceti', 2273], ['Tau Ceti', 2719], ['Gliese 1061', 2940], ['Wolf 1061', 2890], ['Gliese 876', 2490], ['82 G. Eridani', 2730], ['Gliese 581', 2751], ['Lalande 21185', 709], ['Lacaille 9352', 1357], ["Luyten's Star", 1239], ['YZ Ceti', 2178], ['Tau Ceti', 2750], ['Gliese 1061', 2906], ['Wolf 1061', 2932], ['Gliese 876', 2446], ['82 G. 